In [65]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys, os
import time

sys.path.append(os.getcwd())
#import bisect
from bisect import *
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [66]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [67]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    seconds = time.time()
    NODES_EXPANDED = 0
    STATES_REEXPANDED = 0
    DUMMY_NODES = 0
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    expanded = {}

    while frontier:
        # Node extraction with dummy nodes control
        ok = False
        while not ok:
            node = frontier.pop()
            if node.path_cost > reached[node.state].path_cost:
                DUMMY_NODES += 1
            else:
                ok = True
        
        # Reexpansion control
        if node.state in expanded:
            STATES_REEXPANDED += 1
        else:
            expanded[node.state] = True

        # counting expansions  
        NODES_EXPANDED += 1  
        
        # Monitoring expanded states and f-values
        #print(node.state, "f()= ", f(node))
     
        #if goal, show statistics
        if problem.is_goal(node.state):
            #print("Time Taken: ", time.time()-seconds)
            #print("Nodes Expanded: ", NODES_EXPANDED)
            #print("States Reached: ", len(reached))
            #print("Nodes in Frontier: ", len(frontier))
            #print("Solution Cost: ", node.path_cost)
            #print("Solution Length: ", len(node))
            #print("Reexpanded States: ", STATES_REEXPANDED)
            #print("Dummy Nodes: ", DUMMY_NODES)
            return node
        # node expansion
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



In [68]:
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N: int = int(((8*len(lista)+1)**0.5)-1)/2
        #print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

In [69]:
class EstadoTSP:
    def __init__(self, actual, visitadas):
        self.actual = actual
        self.visitadas = visitadas
        lista = visitadas.copy()
        lista.append(actual)
        self.state = tuple(lista)
       # self.hash = hash(self.state)
    def __hash__(self):
        return hash(self.state)
        #return self.hash
    def __eq__(self,other):
        return self.actual == other.actual and self.visitadas == other.visitadas
    def __str__(self):
        return " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]"  

In [70]:
class SymmetricTSP(Problem):
    """El problema consiste en encontrar un camino hamiltoniano en un grafo no dirigido 
    y totalmente conectado, con costes positivos en los ejes. Las ciudades son 0,...,N-1
    La ciudad de partida es la 0"""

    def __init__(self, grafo):
        self.grafo : Grafo = grafo
        self.initial: EstadoTSP = EstadoTSP(0,[])
        #print(self.__dir__initial)

    def actions(self, state):
        """Una accion por cada ciudad no visitada distinta de la actual, si todas visitadas -1"""
        noVisitadas = []
        for c in self.grafo.ciudades: 
            if c not in state.visitadas and c != state.actual:
                noVisitadas.append(c) 
        if len(noVisitadas)==0:
            noVisitadas = [-1]
        return noVisitadas
    
    def result(self, state, action): 
        setVisitadas = list(state.visitadas)
        insort(setVisitadas,state.actual)
        return EstadoTSP(action, setVisitadas)
    
    def is_goal(self, state):        return state.actual == -1

    def action_cost(self, s, a, s1): 
        if (len(s.visitadas)==self.grafo.N-1):
            return self.grafo.Dist[s.actual][0]
        else:
            return self.grafo.Dist[s.actual][a]
        
    def h(self,node):
        return self.h1(node)
    
    def h0(self, node):
        return 0
    
    def h1(self,node):
        # Heruristico que considera los arcos minimos de las ciudades
        # que quedan por abandonar. 
        # No responde a ninguna relajacion, aparentemente

        # Obtener el estado actual desde el nodo
        state = node.state

        # Obtenemos las ciudades no visitadas
        ciudades_no_visitadas = [ciudad for ciudad in self.grafo.ciudades if ciudad not in state.visitadas]

        if not ciudades_no_visitadas:
            return 0 # si no hay ciudades no visitadas, el coste mínimo es 0

        # Crear una lista de costos mínimos desde la ciudad actual a todas las ciudades no visitadas
        suma = 0
        for ciudad in ciudades_no_visitadas:
            costo_arco_minimos = [self.grafo.Dist[ciudad][ciudad_destino] for ciudad_destino in ciudades_no_visitadas if self.grafo.Dist[ciudad][ciudad_destino] > 0]

            # encontrar el costo mínimo usando la función min()
            if (costo_arco_minimos):
                suma = suma + min(costo_arco_minimos)

        return suma
    
    def h2(self, node: Node):
        state: EstadoTSP = node.state

        ciudades_no_visitadas: list[int] = [ciudad for ciudad in self.grafo.ciudades if ciudad not in state.visitadas]
        
        if not ciudades_no_visitadas or len(ciudades_no_visitadas) < 3:
            return 0
     
        non_visited_nor_actual: list[int] = ciudades_no_visitadas.copy()
        if self.initial.actual in non_visited_nor_actual:
            non_visited_nor_actual.remove(self.initial.actual)
        residual_arcs = []
        added_cities = []
        for residual_city in non_visited_nor_actual:
            costo_arco_residual = [self.grafo.Dist[residual_city][ciudad_destino] for ciudad_destino in ciudades_no_visitadas if ciudad_destino not in added_cities and self.grafo.Dist[residual_city][ciudad_destino] > 0]
            residual_arcs.extend(costo_arco_residual)
            added_cities.append(residual_city)

        costo_arco_actual = [self.grafo.Dist[state.actual][ciudad_destino] for ciudad_destino in non_visited_nor_actual if self.grafo.Dist[residual_city][ciudad_destino] > 0]
        residual_arcs.extend(costo_arco_actual)
        residual_arcs.sort()
        return sum(residual_arcs[:int(self.grafo.N - len(node.state.visitadas) + 1)])
    
    def h3(self, node: Node):
        state: EstadoTSP = node.state

        ciudades_no_visitadas_con_arco:list[int] = []
        visitadas_actual:list[int] = state.visitadas.copy()
        visitadas_actual.append(state.actual)

        ciudades_no_visitadas: list[int] = [ciudad for ciudad in self.grafo.ciudades if ciudad not in state.visitadas]

        if not ciudades_no_visitadas or len(ciudades_no_visitadas) < 3:
            return 0

        for ciudad in ciudades_no_visitadas:
            for visitada_o_actual in visitadas_actual:
                if self.grafo.Dist[visitada_o_actual][ciudad]:
                    ciudades_no_visitadas_con_arco.append(ciudad)
                    break
        
        
     
        non_visited_nor_actual: list[int] = ciudades_no_visitadas.copy()
        if self.initial.actual in non_visited_nor_actual:
            non_visited_nor_actual.remove(self.initial.actual)
        residual_arcs = []
        added_cities = []
        for residual_city in non_visited_nor_actual:
            costo_arco_residual = [self.grafo.Dist[residual_city][ciudad_destino] for ciudad_destino in ciudades_no_visitadas if ciudad_destino not in added_cities and self.grafo.Dist[residual_city][ciudad_destino] > 0]
            residual_arcs.extend(costo_arco_residual)
            added_cities.append(residual_city)

        costo_arco_actual = [self.grafo.Dist[state.actual][ciudad_destino] for ciudad_destino in non_visited_nor_actual if self.grafo.Dist[residual_city][ciudad_destino] > 0]
        residual_arcs.extend(costo_arco_actual)
        residual_arcs.sort()
        return sum(residual_arcs[:int(self.grafo.N - len(node.state.visitadas) + 1)])

    def h_mst(self, node):
        state: EstadoTSP = node.state
        visited_cities = set(state.visitadas)
        current_city = state.actual

        # Crear un conjunto de ciudades no visitadas
        unvisited_cities = set(self.grafo.ciudades) - visited_cities

        # Inicializar el árbol de expansión mínimo con el costo de regresar a la ciudad de inicio
        mst_cost = self.grafo.Dist[current_city][0]

        while unvisited_cities:
            min_cost = float('inf')
            next_city = None

            for city in unvisited_cities:
                cost = self.grafo.Dist[current_city][city]
                if cost < min_cost:
                    min_cost = cost
                    next_city = city

            # Agregar el costo mínimo al MST
            mst_cost += min_cost

            # Mover a la siguiente ciudad
            current_city = next_city
            unvisited_cities.remove(next_city)

        # Agregar el costo de regresar a la ciudad de inicio
        mst_cost += self.grafo.Dist[current_city][0]

        return mst_cost

In [71]:
# Datos de instancias en formato EDGE_WEIGHT_TYPE de la TSPLIB

prueba = [ 
    0,
    1, 0,
    6, 2, 0,
    4, 5, 3, 0,
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr12 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0
 ]

gr15 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0
 ]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

instance = SymmetricTSP(Grafo(gr12))



for _ in range(10):
    path_states(astar_search(instance))
    #for w in [0.1,0.5,1,2,3,4,5]:
    #    path_states(weighted_astar_search(instance, weight=w))
    
    


Time Taken:  0.5051732063293457
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.3467133045196533
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.34419918060302734
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.33526086807250977
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.4651179313659668
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.3380422592163086
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.33379292488098145
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.33504414558410645
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.33734607696533203
Nodes Expanded:  2318
Solution Cost:  1799
Time Taken:  0.4538280963897705
Nodes Expanded:  2318
Solution Cost:  1799
